In [1]:
# Parameters
country_name = "Zimbabwe"


# Optimization Problem Time Series

This notebook optimizes an electrci mix based on : 
- an electricity demand time serie
- a PV production time serie
- a wind production time serie

In [6]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from pulp import *
import sys
from utilities import import_excel, optimize_enr_chu, format_load_data
import plotly.graph_objects as go
import plotly.express as px
import pickle
from ren_ninja_api import fetch_and_average_data_ren_ninja, get_regular_coordinates
import plotly.io as pio
import os
import glob
import geopandas as gpd
import matplotlib.pyplot as plt
pio.renderers.default='notebook'

Enter country name for file naming: 

In [22]:
year = 2021
# country_name = 'Canada'
# state_name = 'NU'
country_name = country_name
state_name = state_name
# Directory path
path_input_data = '../input_time_series/'
mode = 'grid'
regions_match = pd.read_excel('match_plexos_iso_codes.xlsx')

regions_match = regions_match[regions_match['Country']==country_name].set_index('region')
plexos_region = regions_match.loc[state_name, 'plexos']
chu_plexos_match = pd.read_excel('../../../DATA/Chu et Hawkes/match_chu_plexos.xlsx', index_col=0)

region_chu = chu_plexos_match[chu_plexos_match['Plexos']==plexos_region].iloc[0, 0]
print(region_chu)
country_codes = pd.read_csv('../countries_codes_and_coordinates_adapted.csv' , sep = ',', index_col = 0)
country_code = country_codes.loc[country_name,'Alpha-3 code'].split( ' ')[1]

NO


In [12]:
# for code in chu_plexos_match.index:
#     if len(chu_plexos_match.loc[code,'Plexos'])==3:
#         chu_plexos_match.loc[code, 'iso_alpha3']= code.split('.')[0]
#         chu_plexos_match.loc[code,'Plexos']=''
#     else: 
#         chu_plexos_match.loc[code, 'iso_alpha3']= code.split('.')[0]
        
# chu_plexos_match.to_excel('../../../DATA/Chu et Hawkes/match_chu_plexos.xlsx')

# Optimisation

### Load Time Series

First we load the time series that will be used in the problem. We use the  ```import_excel``` function used in the wavelet decomposition.

#### Demand

In [13]:
# Demand time serie

dpd = 24 # data per day in the time serie
dpy = 365 # data per year :  cut the leap years to 365 years

ndpd = 24 # new data per day for hourly data (for the interpolation)
signal_length = ndpd * dpy
region_name = regions_match.loc[state_name, 'plexos']
file_name = format_load_data(country_name, state_name = region_name)
Load_ts = import_excel(path_input_data,file_name, 
                                    dpd ,ndpd, dpy, 
                                    interp=True, norm = 'mean') # interpolate data from dpd to ndpd numper of points per day
# Répéter la série temporelle 10 fois
serie_temporelle_rep = np.tile(Load_ts, 10)

# Créer un DataFrame à partir de la série temporelle répétée
Load_ts = serie_temporelle_rep
mean_load = pd.read_excel(path_input_data+file_name).mean().iloc[0]

#### Wind production

In [14]:
# Wind time serie 

dpd = 24 # data per day
dpy = 365 # data per year :  cut the leap years to 365 years

# We interpolate so that we have hourly data
ndpd = 24 # new data per day for hourly data (for the interpolation)
signal_length = ndpd * dpy

path_input_data = '../input_time_series/'
if state_name: 
    file_name = f'{country_name}/{country_code}_{region_chu}_wind_onshore_10y.xlsx'
    if not os.path.exists(path_input_data+f'{country_name}/{country_code}_{region_chu}_wind_onshore_10y.xlsx'):  
        print('here')
        file_name = f'{country_name}/{country_code}_{region_chu}_chu_wind_onshore_averaged.xlsx'
        df = pd.read_excel(path_input_data+file_name, index_col=0)
        
        time_series = pd.concat([df[col] for col in df.columns], ignore_index=True)
        result_df = pd.DataFrame({'Time Series': time_series})

        # Sauvegarder le résultat dans un nouveau fichier Excel
        result_df.to_excel(path_input_data+f'{country_name}/{country_code}_{region_chu}_wind_onshore_10y.xlsx', index=False)
        file_name = f'{country_name}/{country_code}_{region_chu}_wind_onshore_10y.xlsx'
else:
    file_name = f'{country_name}/{country_code}_full_wind_onshore_10y.xlsx'
    if not os.path.exists(path_input_data+f'{country_name}/{country_code}_full_wind_onshore_10y.xlsx'):
        file_name = f'{country_name}/{country_code}_full_chu_wind_onshore_averaged.xlsx'
        df = pd.read_excel(path_input_data+file_name, index_col=0)

        time_series = pd.concat([df[col] for col in df.columns], ignore_index=True)
        result_df = pd.DataFrame({'Time Series': time_series})

        # Sauvegarder le résultat dans un nouveau fichier Excel
        result_df.to_excel(path_input_data+f'{country_name}/{country_code}_full_wind_onshore_10y.xlsx', index=False)
        file_name = f'{country_name}/{country_code}_full_wind_onshore_10y.xlsx'
Wind_ts = import_excel(path_input_data,file_name, 
                                dpd ,ndpd, dpy, 
                                interp=True, norm = None) # interpolate data from dpd to ndpd numper of points per day

mean_wind = pd.read_excel(path_input_data+file_name).mean().iloc[0]

#### PV production

In [15]:
# PV time serie 

dpd = 24 # data per day
dpy = 365 # data per year :  cut the leap years to 365 years

# We interpolate so that we have hourly data
ndpd = 24 # new data per day for hourly data (for the interpolation)
signal_length = ndpd * dpy

path_input_data = '../input_time_series/'
if state_name: 
    file_name = f'{country_name}/{country_code}_{region_chu}_pv_fixed_10y.xlsx'
    if not os.path.exists(path_input_data+file_name):  
        file_name = f'{country_name}/{country_code}_{region_chu}_pv_fixed_aggregated.xlsx'
        df = pd.read_excel(path_input_data+file_name, index_col=0)

        time_series = pd.concat([df[col] for col in df.columns], ignore_index=True)
        result_df = pd.DataFrame({'Time Series': time_series})

        # Sauvegarder le résultat dans un nouveau fichier Excel
        result_df.to_excel(path_input_data+f'{country_name}/{country_code}_{region_chu}_pv_fixed_10y.xlsx', index=False)
        file_name = f'{country_name}/{country_code}_{region_chu}_pv_fixed_10y.xlsx'
else:
    file_name =f'{country_name}/{country_code}_full_pv_fixed_10y.xlsx'
    if not os.path.exists(path_input_data+file_name):
        file_name = f'{country_name}/{country_code}_full_chu_pv_fixed_averaged.xlsx'
        df = pd.read_excel(path_input_data+file_name, index_col=0)

        time_series = pd.concat([df[col] for col in df.columns], ignore_index=True)
        result_df = pd.DataFrame({'Time Series': time_series})

        # Sauvegarder le résultat dans un nouveau fichier Excel
        result_df.to_excel(path_input_data+f'{country_name}/{country_code}_full_pv_fixed_10y.xlsx', index=False)
        
PV_ts = import_excel(path_input_data,file_name, 
                                dpd ,ndpd, dpy, 
                                interp=True, norm = None) # interpolate data from dpd to ndpd numper of points per day

mean_pv = pd.read_excel(path_input_data+file_name).mean().iloc[0]

### Plot Time Series

In [20]:
# colors_dict = {
#     'Wind': 'steelblue',        
#     'PV': 'gold',
#     'Discharge': 'orangered',    
#     'SOC': 'darkgreen',           
#     'Charge': 'purple',
#     'Consumption': 'green',          
#     'Dispatchable': 'crimson',       
#     'Curtailment': 'cyan'    
# }

In [21]:
# # Create a Plotly figure
# fig = go.Figure()

# fig.add_trace(go.Scatter(y=PV_ts, mode='lines', name='PV',marker=dict(color=colors_dict['PV'])))
# fig.add_trace(go.Scatter(y=Load_ts, mode='lines', name='Demand',marker=dict(color=colors_dict['Consumption'])))
# fig.add_trace(go.Scatter(y=Wind_ts, mode='lines', name='Wind',marker=dict(color=colors_dict['Wind'])))
# fig.update_layout(title=f'{country_name} {state_name} 2021', xaxis_title='Hour', yaxis_title='Power normalized')

# # Show the plot
# fig.show()

### Description of the problem

#### Equations:

- **Objective function** : 
  - Minimize dispatchable energy: $ \min(\sum{P_{dispatchable}(t)*dt}) $


- **Node Law** : 
  - $(P_{pv}(t) + P_{wind}(t) + P_{dispatchable}(t) - P_{in\_stock}(t) + P_{out\_stock}(t) = P_{demand}(t) + P_{curt}(t))$


- **State of charge**
  - $SOC(t+1)=SOC(t)+P_{in\_stock}(t) - P_{out\_stock}(t)$

#### Contraintes :
- $E_{wind} + E_{pv} \leq E_{demand}$
- We want a maximum storage size of 10 hours:  $E_{stock} \leq 10$
- Charging and discharging at the same time is not possible. 


### Implementation
#### Decision Variables:
- `x_pv`: Installed capacity for photovoltaic production.
- `x_wind`: Installed capacity for wind production.
- `ts_dispatchable`: Dispatchable production (can be controlled), time serie.
- `p_ch`: Battery charging power, time serie.
- `p_dech`: Battery discharging power, time serie.
- `SOC_ts`: State of charge of the battery, time serie.
- `p_curt`: Curtailment power (lost energy), time serie.
- `dech_active`: Binary variable indicating if the battery is charging or discharging.

### Run the optimization with GUROBI

**If the optimization has already been run, go to the next part where results can be loaded and analysed in plots.**

In [19]:
optimized_parameters=optimize_enr_chu(country_name, Load_ts[:43800], PV_ts[:43800], Wind_ts[:43800], mean_load, state_name = state_name,save_results = True)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2503220
Academic license 2503220 - for non-commercial use only - registered to du___@ethz.ch
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2503220 - for non-commercial use only - registered to du___@ethz.ch
Optimize a model with 175201 rows, 262800 columns and 569397 nonzeros
Model fingerprint: 0x8c65371a
Variable types: 219001 continuous, 43799 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-03, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+05]
  RHS range        [8e-01, 1e+05]
Found heuristic solution: objective 43800.000000
Presolve removed 5 rows and 43805 columns
Presolve time: 4.14s
Presolved: 175196 rows, 218995 columns, 497499 no

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

### Add results to .csv file with all countries

In [21]:
df_new = pd.DataFrame({"Country" : [country_name],"State":[state_name],"iso_alpha":[optimized_parameters['iso_alpha']],
    "mean_load":[optimized_parameters['mean_consumption']],
    "E_dispatch": [optimized_parameters['E_dispatch']],
    "P_dispatch": [optimized_parameters['dispatchable_capacity']],
    "E_destock": [optimized_parameters['E_destock']],
    "P_pv": [optimized_parameters['pv_capacity']],
    "P_wind": [optimized_parameters['wind_capacity']]
})


In [22]:
all_results_file = "results/optimization_results_CHU_world_grid_with_states.csv"
file_exists = os.path.isfile(all_results_file)

df_new.to_csv(all_results_file, mode='a', index=False, header=not file_exists)

print(f"Data have been added to {all_results_file}")    

Data have been added to results/optimization_results_CHU_world_grid_with_states.csv


: 

## Plots

In [13]:
# from plots import plot_ts_optim, plot_pie_energy, plot_storage, plot_stack_production

In [14]:
# plot_ts_optim([optimized_parameters['optimized_pv'], optimized_parameters['optimized_wind'], optimized_parameters['optimized_dispatchable'], optimized_parameters['optimized_p_curt'],np.array(Load_ts) ], ['PV', 'Wind', 'Dispatchable', 'Curtailment', 'Consumption'], country_name, colors_dict = colors_dict,savefig=False)

In [15]:
# plot_storage(optimized_parameters['optimized_charge'], optimized_parameters['optimized_discharge'], optimized_parameters['optimized_stock'], country_name, colors_dict = colors_dict, savefig=False)

In [16]:
# E_wind = optimized_parameters['E_wind']
# E_pv = optimized_parameters['E_pv']
# E_dispatch = optimized_parameters['E_dispatch']

In [17]:
# plot_pie_energy([E_wind, E_pv, E_dispatch], country_name, colors_dict =colors_dict, savefig=False)